In [1]:
import tweepy
import json
import pandas as pd
from pymongo import MongoClient,ASCENDING,TEXT

In [84]:
!pip install -U pymongo

In [2]:
import configparser
# read configs
config = configparser.ConfigParser()
config.read('config.ini')

api_key = config['twitter']['api_key']
api_key_secret = config['twitter']['api_key_secret']

access_token = config['twitter']['access_token']
access_token_secret = config['twitter']['access_token_secret']

In [3]:
Client=MongoClient("mongodb://localhost:27017")



In [4]:
""""# Define Stream Listener Class
class TweepyListener ( tweepy.Stream):
    def on_status (self, status):
        print (status.id_str)
        try:
            tweet_collection.insert_one (status._json)
        except:
            pass
    def on_error (self, status_code):
        if status_code == 420:
            return False"""

'"# Define Stream Listener Class\nclass TweepyListener ( tweepy.Stream):\n    def on_status (self, status):\n        print (status.id_str)\n        try:\n            tweet_collection.insert_one (status._json)\n        except:\n            pass\n    def on_error (self, status_code):\n        if status_code == 420:\n            return False'

In [5]:

# authentication
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

# user tweets
users = ['MehranShakarami']
limit=300
for user in users:
    tweets = tweepy.Cursor(api.user_timeline, screen_name=user, count=200, tweet_mode='extended').items(limit)
    columns = ['User', 'Tweet']
    data = []
    for tweet in tweets:
        data.append([tweet.user.screen_name, tweet.full_text])
# tweets = api.user_timeline(screen_name=user, count=limit, tweet_mode='extended')

# create DataFrame

df1 = pd.DataFrame(data, columns=columns)

print(df1)

               User                                              Tweet
0   MehranShakarami  @marcio_lipe This was a fake copyright claim, ...
1   MehranShakarami  @Ed_FilmBooth This scared me how predictable I...
2   MehranShakarami  The Bungie lawsuit is crazy. I hope this gets ...
3   MehranShakarami  @YouTube @TeamYouTube I just got a fraudulent ...
4   MehranShakarami  @MattyestMatt Today I found out that somebody ...
5   MehranShakarami  I just got a copyright strike and removal requ...
6   MehranShakarami     Continuous vs discrete https://t.co/1X80dYi0XO
7   MehranShakarami  Have you heard about the recent project of Ana...
8   MehranShakarami  @liuyangyang3021 Do you have elevated access? ...
9   MehranShakarami  @elonmusk congrats for the new purchase. Hope ...
10  MehranShakarami  When you finally buy your dream Mac (after 10 ...
11  MehranShakarami  New video is out:\nGet UNLIMITED Tweets by Pyt...
12  MehranShakarami  @TimorOccidental Thanks a lot! I'm so happy th...
13  Me

In [6]:
data=df1.to_dict(orient='records')

In [7]:
data

[{'User': 'MehranShakarami',
  'Tweet': "@marcio_lipe This was a fake copyright claim, but sadly youtube doesn't verify anything before taking down the video. I have filed a counter notification, and the video should get back online in few days 🤞"},
 {'User': 'MehranShakarami',
  'Tweet': '@Ed_FilmBooth This scared me how predictable I was'},
 {'User': 'MehranShakarami',
  'Tweet': 'The Bungie lawsuit is crazy. I hope this gets the attention of YT and they try to improve the CMDA takedown process.'},
 {'User': 'MehranShakarami',
  'Tweet': '@YouTube @TeamYouTube I just got a fraudulent copyright takedown notice on my educational video. All the content is original and it’s shocking to get this removal request. The complainer doesn’t reply to my email and the video is about to be removed. Can somebody help me? https://t.co/WM1GcRaJW6'},
 {'User': 'MehranShakarami',
  'Tweet': '@MattyestMatt Today I found out that somebody put a claim on my video, that is only me coding and explaining. It

In [8]:
db=Client['TweetAnalyze']
cl=db['tweets']


In [9]:
db.cl.insert_many(data)

In [10]:
all_records=db.cl.find()



In [11]:
print(all_records)

In [12]:
for row in all_records:
    print(row)

{'_id': ObjectId('6309ef2599706e42f5970a4d'), 'User': 'MehranShakarami', 'Tweet': "@marcio_lipe This was a fake copyright claim, but sadly youtube doesn't verify anything before taking down the video. I have filed a counter notification, and the video should get back online in few days 🤞"}
{'_id': ObjectId('6309ef2599706e42f5970a4e'), 'User': 'MehranShakarami', 'Tweet': '@Ed_FilmBooth This scared me how predictable I was'}
{'_id': ObjectId('6309ef2599706e42f5970a4f'), 'User': 'MehranShakarami', 'Tweet': 'The Bungie lawsuit is crazy. I hope this gets the attention of YT and they try to improve the CMDA takedown process.'}
{'_id': ObjectId('6309ef2599706e42f5970a50'), 'User': 'MehranShakarami', 'Tweet': '@YouTube @TeamYouTube I just got a fraudulent copyright takedown notice on my educational video. All the content is original and it’s shocking to get this removal request. The complainer doesn’t reply to my email and the video is about to be removed. Can somebody help me? https://t.co/WM

In [13]:
import numpy as np
import re
import nltk
stopwords = nltk.corpus.stopwords.words('english')

def clean_tweet(tweet):
    if type(tweet) == np.float:
        return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)
    temp = re.sub("#[A-Za-z0-9_]+","", temp)
    temp = re.sub(r'http\S+', '', temp)
    temp = re.sub('[()!?]', ' ', temp)
    temp = re.sub('\[.*?\]',' ', temp)
    temp = re.sub("[^a-z0-9]"," ", temp)
    temp = temp.split()
    temp = [w for w in temp if not w in stopwords]
    temp = " ".join(word for word in temp)
    return temp

In [14]:
for row in all_records:
    row=clean_tweet(row)
  

In [15]:

documents = list(db.cl.find())
documents

[{'_id': ObjectId('6309ef2599706e42f5970a4d'),
  'User': 'MehranShakarami',
  'Tweet': "@marcio_lipe This was a fake copyright claim, but sadly youtube doesn't verify anything before taking down the video. I have filed a counter notification, and the video should get back online in few days 🤞"},
 {'_id': ObjectId('6309ef2599706e42f5970a4e'),
  'User': 'MehranShakarami',
  'Tweet': '@Ed_FilmBooth This scared me how predictable I was'},
 {'_id': ObjectId('6309ef2599706e42f5970a4f'),
  'User': 'MehranShakarami',
  'Tweet': 'The Bungie lawsuit is crazy. I hope this gets the attention of YT and they try to improve the CMDA takedown process.'},
 {'_id': ObjectId('6309ef2599706e42f5970a50'),
  'User': 'MehranShakarami',
  'Tweet': '@YouTube @TeamYouTube I just got a fraudulent copyright takedown notice on my educational video. All the content is original and it’s shocking to get this removal request. The complainer doesn’t reply to my email and the video is about to be removed. Can somebody h

In [16]:
df = pd.DataFrame(documents, columns = ['_id','User','Tweet'])
df

,_id,User,Tweet
0,6309ef2599706e42f5970a4d,MehranShakarami,"@marcio_lipe This was a fake copyright claim, ..."
1,6309ef2599706e42f5970a4e,MehranShakarami,@Ed_FilmBooth This scared me how predictable I...
2,6309ef2599706e42f5970a4f,MehranShakarami,The Bungie lawsuit is crazy. I hope this gets ...
3,6309ef2599706e42f5970a50,MehranShakarami,@YouTube @TeamYouTube I just got a fraudulent ...
4,6309ef2599706e42f5970a51,MehranShakarami,@MattyestMatt Today I found out that somebody ...
5,6309ef2599706e42f5970a52,MehranShakarami,I just got a copyright strike and removal requ...
6,6309ef2599706e42f5970a53,MehranShakarami,Continuous vs discrete https://t.co/1X80dYi0XO
7,6309ef2599706e42f5970a54,MehranShakarami,Have you heard about the recent project of Ana...
8,6309ef2599706e42f5970a55,MehranShakarami,@liuyangyang3021 Do you have elevated access? ...
9,6309ef2599706e42f5970a56,MehranShakarami,@elonmusk congrats for the new purchase. Hope ...


In [17]:
Tweets=list()
Tweets=[row['Tweet'] for row in documents]
print(Tweets)

["@marcio_lipe This was a fake copyright claim, but sadly youtube doesn't verify anything before taking down the video. I have filed a counter notification, and the video should get back online in few days 🤞", '@Ed_FilmBooth This scared me how predictable I was', 'The Bungie lawsuit is crazy. I hope this gets the attention of YT and they try to improve the CMDA takedown process.', '@YouTube @TeamYouTube I just got a fraudulent copyright takedown notice on my educational video. All the content is original and it’s shocking to get this removal request. The complainer doesn’t reply to my email and the video is about to be removed. Can somebody help me? https://t.co/WM1GcRaJW6', '@MattyestMatt Today I found out that somebody put a claim on my video, that is only me coding and explaining. It’s a shame how easily people can file these complaints and cause trouble for content creators.', 'I just got a copyright strike and removal request for one of my videos, and funny that I cannot do anythi

In [18]:
conda install pytorch torchvision -c pytorch

^C

CondaError: KeyboardInterrupt


Note: you may need to restart the kernel to use updated packages.


In [20]:
import transformers
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from scipy.special import softmax

In [21]:
RoBERTa= f"cardiffnlp/twitter-roberta-base-sentiment-latest"

In [23]:
tokenizer = AutoTokenizer.from_pretrained(RoBERTa)
model = AutoModelForSequenceClassification.from_pretrained(RoBERTa)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
from transformers import pipeline
classifier=pipeline("sentiment-analysis",model=RoBERTa,tokenizer=tokenizer)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [25]:
results=classifier(df['Tweet'].to_list())

In [26]:
for result in results:
    print(result)

{'label': 'Negative', 'score': 0.712609589099884}
{'label': 'Negative', 'score': 0.8113563060760498}
{'label': 'Negative', 'score': 0.4595453143119812}
{'label': 'Negative', 'score': 0.8951922059059143}
{'label': 'Negative', 'score': 0.9164142608642578}
{'label': 'Negative', 'score': 0.8505436778068542}
{'label': 'Neutral', 'score': 0.8693071603775024}
{'label': 'Positive', 'score': 0.7728056907653809}
{'label': 'Neutral', 'score': 0.6380978226661682}
{'label': 'Positive', 'score': 0.9774083495140076}
{'label': 'Positive', 'score': 0.9298089742660522}
{'label': 'Neutral', 'score': 0.7753016352653503}
{'label': 'Positive', 'score': 0.9884557723999023}
{'label': 'Positive', 'score': 0.9622519016265869}
{'label': 'Positive', 'score': 0.9882391691207886}
{'label': 'Neutral', 'score': 0.7569103837013245}
{'label': 'Neutral', 'score': 0.7994017004966736}
{'label': 'Neutral', 'score': 0.7708052396774292}


In [27]:
#result["label"] for result in results
from operator import itemgetter
df2=df.assign(sentiment=list(map(itemgetter('label'), results)))
df2

,_id,User,Tweet,sentiment
0,6309ef2599706e42f5970a4d,MehranShakarami,"@marcio_lipe This was a fake copyright claim, ...",Negative
1,6309ef2599706e42f5970a4e,MehranShakarami,@Ed_FilmBooth This scared me how predictable I...,Negative
2,6309ef2599706e42f5970a4f,MehranShakarami,The Bungie lawsuit is crazy. I hope this gets ...,Negative
3,6309ef2599706e42f5970a50,MehranShakarami,@YouTube @TeamYouTube I just got a fraudulent ...,Negative
4,6309ef2599706e42f5970a51,MehranShakarami,@MattyestMatt Today I found out that somebody ...,Negative
5,6309ef2599706e42f5970a52,MehranShakarami,I just got a copyright strike and removal requ...,Negative
6,6309ef2599706e42f5970a53,MehranShakarami,Continuous vs discrete https://t.co/1X80dYi0XO,Neutral
7,6309ef2599706e42f5970a54,MehranShakarami,Have you heard about the recent project of Ana...,Positive
8,6309ef2599706e42f5970a55,MehranShakarami,@liuyangyang3021 Do you have elevated access? ...,Neutral
9,6309ef2599706e42f5970a56,MehranShakarami,@elonmusk congrats for the new purchase. Hope ...,Positive


In [28]:
df2.dtypes

_id          object
User         object
Tweet        object
sentiment    object
dtype: object

In [29]:
df2['sentiment']=df2['sentiment'].astype(str)
df2.dtypes

_id          object
User         object
Tweet        object
sentiment    object
dtype: object

In [30]:

list

list

In [31]:
dict=df2['sentiment'].to_dict()

In [32]:
keys_values = dict.items()

new_d = {str(key): str(value) for key, value in keys_values}
new_d

{'0': 'Negative',
 '1': 'Negative',
 '2': 'Negative',
 '3': 'Negative',
 '4': 'Negative',
 '5': 'Negative',
 '6': 'Neutral',
 '7': 'Positive',
 '8': 'Neutral',
 '9': 'Positive',
 '10': 'Positive',
 '11': 'Neutral',
 '12': 'Positive',
 '13': 'Positive',
 '14': 'Positive',
 '15': 'Neutral',
 '16': 'Neutral',
 '17': 'Neutral'}

In [33]:
 for j in new_d.values():
        print(j)

Negative
Negative
Negative
Negative
Negative
Negative
Neutral
Positive
Neutral
Positive
Positive
Neutral
Positive
Positive
Positive
Neutral
Neutral
Neutral


In [60]:
data=df2.values.tolist()

In [88]:

for index, row in df2.iterrows():
    _id=row['_id']
    new_value=row['sentiment']
    db.cl.update_one({'_id':_id},{'$set':{'sentiment':new_value}})
